Model 6 Updates:
- Trying to see if we can get better results with increase of token size 64->128
- Trying increased batch sizes to improve speed at the cost of using more GPU ram, but still only get up to 8.6GB used out of 15 (can increase further!)
- RESULT: Training took slightly longer, but since token size is much larger this is not a bad tradeoff.
- Possibly reducing accuracy with this token size, try reducing it again.


- ACCURACY: .78 (Batch size 128)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from tensorflow.keras import mixed_precision

In [ ]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d176afde-c5e0-55b5-d760-1b94d9e1a920)


In [ ]:
# Enable mixed precision training
mixed_precision.set_global_policy('mixed_float16')

# Read in CSV Data for Twitter Sentiment Analysis
df = pd.read_csv("twitter_sentiment_data.csv")

# Get sentences and labels as dataframes
sentences = df["message"].to_numpy()
labels = df["sentiment"].to_numpy()

# Remove -1
labels[labels == -1] = 3

# One-hot encode the labels
num_classes = 4
labels = tf.keras.utils.to_categorical(labels, num_classes)

# Split into train and test
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=1)

# Initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(list(train_sentences), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_sentences), truncation=True, padding=True, max_length=128)

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

# Initialize the model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_classes)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Create EarlyStopping instance
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Fit the model
model_history = model.fit(train_dataset.shuffle(1000).batch(128),
                          epochs=5,
                          validation_data=test_dataset.batch(128),
                          callbacks=[early_stopping])

# Evaluate the model on the test set
model_results = model.evaluate(test_dataset.batch(128))
print(f"Loss: {model_results[0]}, Accuracy: {model_results[1]}")

# Make Predictions
model_pred = model.predict(test_dataset.batch(128))

# Convert predictions to labels
model_pred_labels = np.argmax(model_pred.logits, axis=1)

# Print the confusion matrix
cm = confusion_matrix(np.argmax(test_labels, axis=1), model_pred_labels)
print(cm)

# Print the classification report
cr = classification_report(np.argmax(test_labels, axis=1), model_pred_labels)
print(cr)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/5
275/275 [==============================] - 241s 757ms/step - loss: 0.7696 - accuracy: 0.6928 - val_loss: 0.6020 - val_accuracy: 0.7590
Epoch 2/5
275/275 [==============================] - 185s 674ms/step - loss: 0.4796 - accuracy: 0.8146 - val_loss: 0.5657 - val_accuracy: 0.7812
Epoch 3/5
275/275 [==============================] - 183s 664ms/step - loss: 0.3026 - accuracy: 0.8898 - val_loss: 0.6570 - val_accuracy: 0.7817
Epoch 4/5
275/275 [==============================] - 183s 664ms/step - loss: 0.1737 - accuracy: 0.9411 - val_loss: 0.7736 - val_accuracy: 0.7809
Epoch 5/5
69/69 [==============================] - 15s 216ms/step - loss: 0.5657 - accuracy: 0.7812
Loss: 0.5657070279121399, Accuracy: 0.7812038064002991
69/69 [==============================] - 17s 213ms/step
[[ 976  385  119   84]
 [ 433 3730  327   42]
 [  35  153 1669    9]
 [ 174  129   33  491]]
              precision    recall  f1-score   support

           0       0.60      0.62      0.61      1564
       